In [1]:
# Import the QP solver and Problem classes
import numpy as np
from lava.lib.optimization.problems.problems import QP
from lava.lib.optimization.solvers.qp.solver import QPSolver
from lava.magma.core.run_conditions import RunSteps
from lava.magma.core.run_configs import Loihi1SimCfg
from lava.lib.optimization.solvers.qp.models import (
    ConstraintCheck,
    GradientDynamics,
)

### Small Problem Floating point w/o sigma delta


In [2]:
P = np.array([[30, 10, 2], [10, 3, 2], [10, 20, 5]])
q = np.array([[1, 2, 1]]).T
tau = 10

## setting up the LASSO problem
Q = P.T@P
p = -P.T@q
A = np.array([[-1, -1,  1], 
              [-1,  1, -1],
              [-1,  1,  1],
              [ 1, -1, -1],
              [ 1, -1,  1],
              [ 1,  1, -1],
              [ 1,  1,  1],
              [-1, -1, -1],
             ])
k = tau*np.ones((1,8)).T

## Learning constants for the solver
alpha_d, beta_g = 10000, 10000

pre_mat_Q = np.diag(1/np.linalg.norm(Q, axis=1))
Q_pre = pre_mat_Q@Q@pre_mat_Q
p_pre = pre_mat_Q@p
alpha = 100

pre_mat_A = np.diag(1/np.linalg.norm(A, axis=1))
A_pre = pre_mat_A@A@pre_mat_Q
k_pre = pre_mat_A@k
beta = 1

In [5]:
init_sol = np.random.rand(Q_pre.shape[0], 1)
iterations = 1000
alpha_d, beta_g = 10000, 10000
alpha, beta = 0.0005, 1

# Initialize the ConstraintCheck Process of the QP
ConsCheck = ConstraintCheck(constraint_matrix=A_pre,
                            constraint_bias=k_pre,
                            )

# Initialize the Gradient Dynamics process of the QP
GradDyn = GradientDynamics(hessian=Q_pre, 
                           constraint_matrix_T=A_pre.T,
                           qp_neurons_init=init_sol, 
                           grad_bias=p_pre, alpha=alpha, 
                           beta=beta, 
                           alpha_decay_schedule=alpha_d,
                           beta_growth_schedule=beta_g,
                           )

In [6]:
# Connect the two hierarchical processes to form core QP 
# solver
GradDyn.a_out.connect(ConsCheck.s_in)
ConsCheck.a_out.connect(GradDyn.s_in)

# Enable select_sub_proc_model to run subprocess models
GradDyn.run(condition=RunSteps(num_steps=iterations),
            run_cfg=Loihi1SimCfg(select_sub_proc_model=True)
           )

# profiling counters
tot_synops = GradDyn.vars.qC_synops.get() + GradDyn.vars.cN_synops.get() + GradDyn.vars.sN_synops.get() \
            + ConsCheck.vars.cNeur_synops.get() + ConsCheck.vars.cD_synops.get() 
tot_spikeops = GradDyn.vars.qC_spikeops.get() + GradDyn.vars.cN_spikeops.get() + GradDyn.vars.sN_spikeops.get() \
            + ConsCheck.vars.cNeur_spikeops.get() + ConsCheck.vars.cD_spikeops.get() 
tot_neurops = GradDyn.vars.qC_neurops.get() + GradDyn.vars.cN_neurops.get() + GradDyn.vars.sN_neurops.get() \
            + ConsCheck.vars.cNeur_neurops.get() + ConsCheck.vars.cD_neurops.get() 

# get solution of kth iteraation of MPC 
pre_sol_k = GradDyn.vars.qp_neuron_state.get()

# stop process execution
GradDyn.stop()

# postconditioning to get actual solution
sol_k = pre_mat_Q@pre_sol_k

In [7]:
print("For kth iteration number of operations per iteration \n Neuron Ops: {} \n Syn Ops: {} \n Spike Ops: {}\n".format(tot_neurops/iterations, 
                                                                                     tot_synops/iterations, tot_spikeops/iterations))

For kth iteration number of operations per iteration 
 Neuron Ops: [[3.]] 
 Syn Ops: [[66.]] 
 Spike Ops: [[14.]]



## 100+ Dim MPC problem w/o sigma delta floating point

## 5000+ Dim MPC problem w/o sigma delta floating point